# Investigating particle swarm optimisation in solving the Travelling Salesman Problem

## Abstract

Particle Swarm Optimisation (PSO) is a computational technique used to optimise a problem by iteratively improving the particles based on eachother, taking inspiration from nature, such as from birds in flight. PSO has many applications, from robotics to networks.

The Travelling Salesman Problem is a famous combinatorial optimisation problem which asks for shortest route by which a salesman can visit a set of locations and return home. It is NP-hard, meaning that there is no efficient algorithm that can solve it in polynomial time. Hence, approximation methods such as PSO are necessary.

In this notebook I investigate using PSO in solving instances of the Travelling Salesman Problem, taken from the TSPLIB dataset. A variety of instances, from small (burma14) to medium (att48) to large (lin105) were used in testing it.

I developed 3 variants of the optimiser:
<li>An initial approach utilising random swaps, the idea taken from this paper https://github.com/marcoscastro/tsp_pso/blob/master/references/paper.pdf</li>
<li>An improved approach using velocity based swaps and greedy initialisation</li>
<li>A third approach using segment reinsertion inspired by genetic mutations, inspired by https://www.tandfonline.com/doi/full/10.1080/23311835.2015.1048581#d1e1432.</li>

The results show that segment reinsertion is significantly better in performance than the other approaches taken, and consistently provides optimal solutions, on par with and if not better than the results found in https://www.tandfonline.com/doi/full/10.1080/23311835.2015.1048581#d1e1432.

## Learning Objectives

## Contents

## Comparison with Other Approaches